In [1]:
import pandas as pd
import numpy as np
import google_analytics_reporting_api as ga
import json as json
import datetime

In [2]:
# Input
Start_Date='2017-10-22'
End_Date=str(datetime.datetime.now().date()-datetime.timedelta(days=1)) # Yesterday
writer_Category="/home/jian/Projects/Tone_Networks/Output/Tone_Network_GA_Category_Data_From_2017-10-22_To_"+End_Date+".xlsx"

In [3]:
writer_Category=pd.ExcelWriter(writer_Category, engine='xlsxwriter')
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Tone_Networks/GA/Tone Networks - prod-00ada014c4ba.json')
general_matric=['ga:sessions','ga:users','ga:bounces','ga:bounceRate','ga:pageviews','ga:uniquePageviews','ga:totalEvents','ga:uniqueEvents']

In [4]:
def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].astype(int)
    if 'ga:users' in df.columns:
        df['ga:users']=df['ga:users'].astype(int)
    if 'ga:bounces' in df.columns:
        df['ga:bounces']=df['ga:bounces'].astype(int)
    if 'ga:bounceRate' in df.columns:
        df['ga:bounceRate']=round(df['ga:bounceRate'].astype(float)/100,4)
    if 'ga:pageviews' in df.columns:
        df['ga:pageviews']=df['ga:pageviews'].astype(int)
    if 'ga:uniquePageviews' in df.columns:
        df['ga:uniquePageviews']=df['ga:uniquePageviews'].astype(int)
    if 'ga:totalEvents' in df.columns:
        df['ga:totalEvents']=df['ga:totalEvents'].astype(int)
    if 'ga:uniqueEvents' in df.columns:
        df['ga:uniqueEvents']=df['ga:uniqueEvents'].astype(int)
    if 'ga:goalCompletionsAll' in df.columns:
        df['ga:goalCompletionsAll']=df['ga:goalCompletionsAll'].astype(int)
    if 'ga:dateHourMinute' in df.columns:
        df['ga:dateHourMinute']=df['ga:dateHourMinute'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8]+" "+x[8:10]+":"+x[10:12])
    if 'ga:date' in df.columns:
        df['ga:date']=df['ga:date'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8])
    
    return df

In [5]:
Start_Minute=datetime.datetime.strptime(Start_Date,'%Y-%m-%d')
End_Minute=datetime.datetime.strptime(str(datetime.datetime.now().date()),'%Y-%m-%d')
All_Minutes=pd.date_range(start=Start_Minute,end=End_Minute,freq="Min")


In [6]:
Start_Minute=datetime.datetime.strptime(Start_Date,'%Y-%m-%d')
End_Minute=datetime.datetime.strptime(str(datetime.datetime.now().date()),'%Y-%m-%d')
All_Minutes=pd.date_range(start=Start_Minute,end=End_Minute,freq="Min")

def Add_Zero_Minute(df):

    df['dateHourMinute']=df['dateHourMinute'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M'))
    Empty_Time_Df=pd.DataFrame(columns=df.columns,index=range(len(All_Minutes)))
    Empty_Time_Df['dateHourMinute']=All_Minutes
    Time_Df_Notin=Empty_Time_Df[~Empty_Time_Df['dateHourMinute'].isin(df['dateHourMinute'])]
    df=df.append(Time_Df_Notin)
    df.reset_index(inplace=True)
    del (df['index'])
    df=df.sort_values('dateHourMinute')
    df.reset_index(inplace=True)
    del (df['index'])    
    return df  

In [7]:
Event_Mapping=pd.read_excel("/home/jian/Projects/Tone_Networks/event map.xlsx")
Event_Mapping.columns=['eventCategory','eventCategory_New']
Event_Mapping['eventCategory']=Event_Mapping['eventCategory'].apply(lambda x:x[14:len(x)])
Event_Mapping['eventCategory_New']=Event_Mapping['eventCategory_New'].apply(lambda x:x[14:len(x)])

In [8]:
# Event_Minute
Event_Time=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:dateHourMinute','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Event_Time=num_func_trans(Event_Time)
Event_Time=Event_Time[['ga:dateHourMinute','ga:eventCategory','ga:eventAction','ga:eventLabel']+['ga:totalEvents','ga:uniqueEvents']]
Event_Time.rename(columns=lambda x:x[3:len(x)],inplace=True)
Event_Time=Add_Zero_Minute(Event_Time)
# Event_Time['eventAction']=Event_Time['eventAction'].replace(0,"No Event")
# Event_Time['eventCategory']=Event_Time['eventCategory'].replace(0,"No Event")
# Event_Time['eventLabel']=Event_Time['eventLabel'].replace(0,"No Event")
Event_Time_Mapped=pd.merge(Event_Time,Event_Mapping,on="eventCategory",how="left")
Event_Time_Mapped.to_excel(writer_Category,sheet_name="Event_all_label",index=False)

Event_Time_Category=Event_Time[['dateHourMinute','eventCategory','totalEvents','uniqueEvents']]
Event_Time_Agg=pd.merge(Event_Time_Category,Event_Mapping,on="eventCategory",how="left")
Event_Time_Agg=Event_Time_Agg.groupby(['dateHourMinute','eventCategory_New'])['totalEvents','uniqueEvents'].sum()
Event_Time_Agg=Event_Time_Agg.reset_index()
Event_Time_Agg=Event_Time_Agg.sort_values(['dateHourMinute','eventCategory_New'])
Event_Time_Agg.to_excel(writer_Category,sheet_name="Event_Category",index=False)


In [9]:
# Minute Page Path
Dimension=['ga:dateHourMinute','ga:pagePath']
Minute_Page_Path=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Minute_Page_Path=num_func_trans(Minute_Page_Path)
Minute_Page_Path=Minute_Page_Path[Dimension+general_matric]

Minute_Page_Path.rename(columns=lambda x:x[3:len(x)],inplace=True)
Minute_Page_Path=Add_Zero_Minute(Minute_Page_Path)
Minute_Page_Path.to_excel(writer_Category,"Minute_Page_Path",index=False)


In [10]:
# Minute MediaSource
Dimension=['ga:dateHourMinute','ga:sourceMedium']
Minute_SourceMedium=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Minute_SourceMedium=num_func_trans(Minute_SourceMedium)
Minute_SourceMedium=Minute_SourceMedium[Dimension+general_matric]

Minute_SourceMedium.rename(columns=lambda x:x[3:len(x)],inplace=True)
Minute_SourceMedium=Add_Zero_Minute(Minute_SourceMedium)
Minute_SourceMedium.to_excel(writer_Category,"Minute_SourceMedium",index=False)


In [11]:
# Minute Metro_City
Dimension=['ga:dateHourMinute','ga:metro','ga:city']
Minute_Metro_City=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Minute_Metro_City=num_func_trans(Minute_Metro_City)
Minute_Metro_City=Minute_Metro_City[Dimension+general_matric]

Minute_Metro_City.rename(columns=lambda x:x[3:len(x)],inplace=True)
Minute_Metro_City=Add_Zero_Minute(Minute_Metro_City)
Minute_Metro_City.to_excel(writer_Category,"Minute_Metro_City",index=False)


In [12]:
def f(x):
    d = {}
    d['pagePath_list'] = list(x['pagePath'])
    d['pagePath_sessions'] = x['sessions'].sum()
    d['pagePath_users'] = x['users'].sum()
    d['pagePath_bounces'] = x['bounces'].sum()
    return pd.Series(d, index=['pagePath_list','pagePath_sessions','pagePath_users','pagePath_bounces'])

Minute_Page_Path_Agg=Minute_Page_Path.groupby('dateHourMinute').apply(f)
Minute_Page_Path_Agg.reset_index(inplace=True)

In [13]:
def f(x):
    d = {}
    d['sourceMedium_list'] = list(x['sourceMedium'])
    d['sourceMedium_sessions'] = x['sessions'].sum()
    d['sourceMedium_users'] = x['users'].sum()
    d['sourceMedium_bounces'] = x['bounces'].sum()
    return pd.Series(d, index=['sourceMedium_list','sourceMedium_sessions','sourceMedium_users','sourceMedium_bounces'])
 
Minute_SourceMedium_Agg=Minute_SourceMedium.groupby('dateHourMinute').apply(f)
Minute_SourceMedium_Agg.reset_index(inplace=True)

In [14]:
def f(x):
    d = {}
    d['metro_list'] = list(x['metro'])
    d['city_list'] = list(x['city'])
    d['geo_sessions'] = x['sessions'].sum()
    d['geo_users'] = x['users'].sum()
    d['geo_bounces'] = x['bounces'].sum()
    return pd.Series(d, index=['metro_list','city_list','geo_sessions','geo_users','geo_bounces'])

Minute_Metro_City_Agg=Minute_Metro_City.groupby('dateHourMinute').apply(f)
Minute_Metro_City_Agg.reset_index(inplace=True)

In [15]:
combined_df=pd.merge(Minute_Page_Path_Agg,Minute_Metro_City_Agg,on='dateHourMinute',how='outer')
combined_df=pd.merge(combined_df,Minute_SourceMedium_Agg,on='dateHourMinute',how='outer')


In [16]:
Event_Time_combined=Event_Time_Mapped[['dateHourMinute','eventCategory_New','eventAction','eventLabel','totalEvents']]
Event_Time_combined.reset_index(inplace=True)
Event_Time_combined['Event_Category_Action_Lable']=Event_Time_combined['eventCategory_New']+" | "+\
                                                    Event_Time_combined['eventAction']+" | "+Event_Time_combined['eventLabel']
Event_Time_combined['Event_Category_Action_Lable']=Event_Time_combined['Event_Category_Action_Lable'].fillna('nan')
Event_Time_combined=Event_Time_combined[['dateHourMinute','Event_Category_Action_Lable','totalEvents']]    

Event_Time_combined_wide=Event_Time_combined.pivot(index='dateHourMinute',columns='Event_Category_Action_Lable',values='totalEvents')
del Event_Time_combined_wide['nan']
Event_Time_combined_wide.reset_index(inplace=True)


In [17]:
combined_df=pd.merge(combined_df,Event_Time_combined_wide,on='dateHourMinute',how='outer')
combined_df.to_excel(writer_Category,"combined_data",index=False)


In [18]:
writer_Category.save()